# Out of Band Protocol - Receiver

The out of band protocol allows agents to exchange messages without requiring a DIDComm channel. This can be used to establish a connection, request a presentation or issue a credential. 

The RFC is described [here](https://github.com/hyperledger/aries-rfcs/tree/master/features/0434-outofband)

Begin in the [sender notebook](http://localhost:8888/notebooks/Part%208%20-%20Out%20of%20Band%20Protocol.ipynb).

## 5. Initialise the Controller

In [2]:
%autoawait
import time
import asyncio

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

IPython autoawait is `on`, and set to use `asyncio`


## 6. Configure Connection Listeners

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connections_handler(payload):
    print("Connection Webhook : ", payload)
    
connections_listener = {
    "topic": "connections",
    "handler": connections_handler
}


agent_controller.register_listeners([connections_listener], defaults=True)

## 7. Receive Out of Band Invitation

**You should have copied this from the Sender notebook (Step 3)**

In [6]:
import json
oob_invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/out-of-band/1.0/invitation', '@id': '4a611f31-396a-4e94-bdfb-820d81bde452', 'request~attach': [], 'handshake_protocols': ['https://didcomm.org/connections/1.0/invitation', 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation'], 'label': 'Alice', 'service': [{'id': '#inline', 'type': 'did-communication', 'recipientKeys': ['did:key:z6MkuU5tbz5eQ5e5w2NCB1rHFWX5dbMvu8G1yoiY7M9Zd7cH'], 'routingKeys': [], 'serviceEndpoint': 'http://172.17.0.1:8020'}]}

response = await agent_controller.oob.receive_invitation(oob_invite)
print(response)

{'request_id': '54503fbb-0f20-475b-b014-e6f411e85dea', 'connection_id': '641c865f-0bfb-4ccd-bc26-e5328a1b3f05', 'their_label': 'Alice', 'accept': 'auto', 'created_at': '2020-10-30 11:28:20.655988Z', 'invitation_key': 'G1pr1jqD4Y9cpXXVVStSQQy5p265VF1fHnocH5BYhtpu', 'my_did': 'HdesGn1FTbpZjq3Sg2U8hQ', 'routing_state': 'none', 'updated_at': '2020-10-30 11:28:20.673566Z', 'initiator': 'external', 'state': 'request', 'invitation_mode': 'once'}


## 8. Get Connection

In [8]:
connection_id = response["connection_id"]
print(connection_id)

connection = await agent_controller.connections.get_connection(connection_id)

print(connection)
print("Connection State : ", connection["state"])

641c865f-0bfb-4ccd-bc26-e5328a1b3f05
{'request_id': '54503fbb-0f20-475b-b014-e6f411e85dea', 'connection_id': '641c865f-0bfb-4ccd-bc26-e5328a1b3f05', 'their_label': 'Alice', 'accept': 'auto', 'created_at': '2020-10-30 11:28:20.655988Z', 'invitation_key': 'G1pr1jqD4Y9cpXXVVStSQQy5p265VF1fHnocH5BYhtpu', 'my_did': 'HdesGn1FTbpZjq3Sg2U8hQ', 'routing_state': 'none', 'updated_at': '2020-10-30 11:28:20.673566Z', 'initiator': 'external', 'state': 'request', 'invitation_mode': 'once'}


## 9. Continue in the Sender [Notebook](http://localhost:8888/notebooks/Part%208%20-%20Out%20of%20Band%20Protocol.ipynb)

You will need to accept the request and send a trust ping before this connection moves to the active state.